In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
data = ['a', 'a', 'b','b', 'a', 'a','a','a','b','b','b','b','b', 'a','a','b','b','a','b','b']
test_df = pd.DataFrame(data, columns = ['product'])
test_df.head()

,product
0,a
1,a
2,b
3,b
4,a


In [4]:
from statsmodels.sandbox.stats.runs import runstest_1samp

test_df.loc[test_df['product'] == 'a', 'product'] = 1
test_df.loc[test_df['product'] == 'b', 'product'] = 0

runstest_1samp(test_df['product'], cutoff = 0.5, correction = True)
# ( Runs 테스트의 통계량-1.1144881152070183,  p-value  0.26506984027306035) 유의하지 않으므로 연관이 없다.

(-1.1144881152070183, 0.26506984027306035)

#Apriori 알고리즘

In [6]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder

dataset = [['Apple', 'Beer', 'Rice', 'Chicken'],
           ['Apple', 'Beer', 'Rice'],
           ['Apple', 'Beer'],
           ['Apple', 'Bananas'],
           ['Milk', 'Beer', 'Rice', 'Chicken'],
           ['Milk', 'Beer', 'Rice'],
           ['Milk', 'Beer'],
           ['Apple', 'Bananas']]

te = TransactionEncoder()
te_ary = te.fit_transform(dataset)
print(te.columns_)
te_ary

['Apple', 'Bananas', 'Beer', 'Chicken', 'Milk', 'Rice']


array([[ True, False,  True,  True, False,  True],
       [ True, False,  True, False, False,  True],
       [ True, False,  True, False, False, False],
       [ True,  True, False, False, False, False],
       [False, False,  True,  True,  True,  True],
       [False, False,  True, False,  True,  True],
       [False, False,  True, False,  True, False],
       [ True,  True, False, False, False, False]])

In [7]:
df = pd.DataFrame(te_ary, columns = te.columns_)
df

,Apple,Bananas,Beer,Chicken,Milk,Rice
0,True,False,True,True,False,True
1,True,False,True,False,False,True
2,True,False,True,False,False,False
3,True,True,False,False,False,False
4,False,False,True,True,True,True
5,False,False,True,False,True,True
6,False,False,True,False,True,False
7,True,True,False,False,False,False


In [8]:
from mlxtend.frequent_patterns import apriori
apriori(df, min_support = 0.6, use_colnames = True) # 최소지지도가 0.6이상만 추출


,support,itemsets
0,0.625,(Apple)
1,0.750,(Beer)


In [9]:
frequent_itemsets = apriori(df, min_support = 0.3, use_colnames = True) #최소지지도 0.3이상 규칙의 길이를 파악하려면 lambda 함수 사용
frequent_itemsets['lenght'] = frequent_itemsets['itemsets'].apply(lambda x : len(x))
frequent_itemsets  #support는 발생빈도를 나타냄  , 0.375	(Apple, Beer)	는 연관성정도를 나타냄

,support,itemsets,lenght
0,0.625,(Apple),1
1,0.750,(Beer),1
2,0.375,(Milk),1
3,0.500,(Rice),1
4,0.375,"(Apple, Beer)",2
5,0.375,"(Milk, Beer)",2
6,0.500,"(Rice, Beer)",2


In [14]:
df = pd.read_csv('/content/drive/MyDrive/datasets/groceries.csv')
df.head()

,Unnamed: 0,"citrus fruit,semi-finished bread,margarine,ready soups"
0,0,"tropical fruit,yogurt,coffee"
1,1,whole milk
2,2,"pip fruit,yogurt,cream cheese,meat spreads"
3,3,"other vegetables,whole milk,condensed milk,lon..."
4,4,"whole milk,butter,yogurt,rice,abrasive cleaner"


In [20]:
df.drop(['Unnamed: 0'], axis = 1, inplace= True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9834 entries, 0 to 9833
Data columns (total 1 columns):
 #   Column                                                  Non-Null Count  Dtype 
---  ------                                                  --------------  ----- 
 0   citrus fruit,semi-finished bread,margarine,ready soups  9834 non-null   object
dtypes: object(1)
memory usage: 77.0+ KB


In [21]:
df_split = df['citrus fruit,semi-finished bread,margarine,ready soups'].str.split(',',expand = True)
df_split.values

array([['tropical fruit', 'yogurt', 'coffee', ..., None, None, None],
       ['whole milk', None, None, ..., None, None, None],
       ['pip fruit', 'yogurt', 'cream cheese', ..., None, None, None],
       ...,
       ['chicken', 'citrus fruit', 'other vegetables', ..., None, None,
        None],
       ['semi-finished bread', 'bottled water', 'soda', ..., None, None,
        None],
       ['chicken', 'tropical fruit', 'other vegetables', ..., None, None,
        None]], dtype=object)

In [25]:
df_split_ary = df_split.values
groceries = []
for i in range(len(df_split_ary)):
  temp = list(filter(None, df_split_ary[i]))
  groceries.append(temp)

groceries[1:10]

[['whole milk'],
 ['pip fruit', 'yogurt', 'cream cheese', 'meat spreads'],
 ['other vegetables',
  'whole milk',
  'condensed milk',
  'long life bakery product'],
 ['whole milk', 'butter', 'yogurt', 'rice', 'abrasive cleaner'],
 ['rolls/buns'],
 ['other vegetables',
  'UHT-milk',
  'rolls/buns',
  'bottled beer',
  'liquor (appetizer)'],
 ['potted plants'],
 ['whole milk', 'cereals'],
 ['tropical fruit',
  'other vegetables',
  'white bread',
  'bottled water',
  'chocolate']]

In [27]:
from mlxtend.preprocessing import TransactionEncoder
import pandas as pd
te = TransactionEncoder()
groceries_tr = te.fit(groceries).transform(groceries)
groceries_tr = pd.DataFrame(groceries_tr, columns = te.columns_)
groceries_tr

,Instant food products,UHT-milk,abrasive cleaner,artif. sweetener,baby cosmetics,baby food,bags,baking powder,bathroom cleaner,beef,...,turkey,vinegar,waffles,whipped/sour cream,whisky,white bread,white wine,whole milk,yogurt,zwieback
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
4,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9829,False,False,False,False,False,False,False,False,False,True,...,False,False,False,True,False,False,False,True,False,False
9830,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
9831,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
9832,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [30]:
from mlxtend.frequent_patterns import apriori
#지지도가 5%이상인 빈번 항목집합 탐색

groceries_ap = apriori(groceries_tr, min_support = 0.01, use_colnames = True)

groceries_ap


,support,itemsets
0,0.033455,(UHT-milk)
1,0.017694,(baking powder)
2,0.052471,(beef)
3,0.033252,(berries)
4,0.026032,(beverages)
...,...,...
328,0.011999,"(whole milk, tropical fruit, root vegetables)"
329,0.014541,"(whole milk, yogurt, root vegetables)"
330,0.010474,"(soda, whole milk, yogurt)"
331,0.015152,"(whole milk, tropical fruit, yogurt)"


In [32]:
from mlxtend.frequent_patterns import association_rules # association_rules 은 한번에 많은 규칙을 파악할 수 있음
#신뢰도가 0.3 이상인 빈번항목집합 탐색
association_rules(groceries_ap, metric = 'confidence', min_threshold = 0.3)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(beef),(other vegetables),0.052471,0.193512,0.019727,0.375969,1.942869,0.009574,1.292384,0.512171
1,(beef),(root vegetables),0.052471,0.109010,0.017389,0.331395,3.040058,0.011669,1.332612,0.708220
2,(beef),(whole milk),0.052471,0.255542,0.021253,0.405039,1.585018,0.007844,1.251271,0.389532
3,(berries),(other vegetables),0.033252,0.193512,0.010270,0.308869,1.596118,0.003836,1.166909,0.386326
4,(berries),(whole milk),0.033252,0.255542,0.011796,0.354740,1.388187,0.003299,1.153733,0.289254
...,...,...,...,...,...,...,...,...,...,...
120,"(soda, yogurt)",(whole milk),0.027354,0.255542,0.010474,0.382900,1.498382,0.003484,1.206381,0.341968
121,"(whole milk, tropical fruit)",(yogurt),0.042302,0.139516,0.015152,0.358173,2.567255,0.009250,1.340679,0.637444
122,"(tropical fruit, yogurt)",(whole milk),0.029286,0.255542,0.015152,0.517361,2.024564,0.007668,1.542474,0.521334
123,"(whole milk, whipped/sour cream)",(yogurt),0.032235,0.139516,0.010881,0.337539,2.419361,0.006383,1.298921,0.606209


In [33]:
rules = association_rules(groceries_ap, metric = 'lift', min_threshold = 1)
rules['antecedent_len'] = rules['antecedents'].apply(lambda x : len(x))
rules[(rules['antecedent_len'] >= 2 ) & (rules['confidence'] >=0.4) & (rules['lift'] >= 3)]


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric,antecedent_len
419,"(citrus fruit, root vegetables)",(other vegetables),0.017694,0.193512,0.010372,0.586207,3.029300,0.006948,1.949012,0.681957,2
492,"(tropical fruit, root vegetables)",(other vegetables),0.021049,0.193512,0.012304,0.584541,3.020692,0.008231,1.941197,0.683334,2
